<a href="https://colab.research.google.com/github/vhcarrillo/CPY16S/blob/main/CPY16S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is for the analysis of 16S amplicon sequencing form cenotes in the Yucatan Peninsula

Fist, import the dataset from a repo in Github

In [ ]:
!git clone https://github.com/vhcarrillo/CPY16S.git

Cloning into 'CPY16S'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 13
Unpacking objects: 100% (16/16), done.


The original files listed in the repo are:

In [ ]:
%ls CPY16S/

CPY16S.ipynb  paper_sep18.ipynb  rep-seqs-cenotes.qza  table-freq-cenotes.qza
metadata.tsv  README.md          setup_qiime2.py       table-freq-cenotes.qzv


*   metadata.tsv contains the metadata for the sites of study
* The raw data were procesed in by Seth in order to combine the files obteined form the sequencer because some samples were sequenced first and the others later. After combined the data, were denoised by dada2 according qiime2 documentation. The artifacts (files) obtained were:
  * rep-seqs-cenotes.qza
  * table-freq-cenotes.qza
  * table-freq-cenotes.qzv



In the repository is there a file to install qiime2 named setup_qiime2.py got it from https://github.com/Gibbons-Lab/isb_course_2021/blob/main/setup_qiime2

In [ ]:
%run CPY16S/setup_qiime2.py

[15:47:53] 🐍 Downloading miniconda...                                     ]8;id=223601;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=677887;file:///content/CPY16S/setup_qiime2.py#39\39]8;;\

[15:47:54] 🐍 Done.                                                        ]8;id=422090;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=874317;file:///content/CPY16S/setup_qiime2.py#45\45]8;;\

           🐍 Installing miniconda...                                      ]8;id=112589;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=275699;file:///content/CPY16S/setup_qiime2.py#39\39]8;;\

[15:48:15] 🐍 Installed miniconda to `/usr/local` 🐍                       ]8;id=257755;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=163544;file:///content/CPY16S/setup_qiime2.py#45\45]8;;\

           🔍 Downloading Qiime 2 package list...                          ]8;id=889604;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=221854;file:///content/CPY16S/setup_qiime2.py#39\39]8;;\

           🔍 Done.                                                        ]8;id=30976;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=462258;file:///content/CPY16S/setup_qiime2.py#45\45]8;;\

           🔍 Installing Qiime 2. This may take a little bit.              ]8;id=914520;file:///content/CPY16S/setup_qiime2.py\setup_qiime2.py]8;;\:]8;id=215377;file:///content/CPY16S/setup_qiime2.py#39\39]8;;\
            🕐                                                                               

As the artifacts have have all the samples sequenced and in three of them there are no sequences it will remove.

| Sample ID |	Sequence Count |
| --------- | -------------- |
| YB03 |	60,504 |
| YB08 |	44,902 |
| YBO4 |	43,221 |
| YB13 |	41,795 |
| MX.01 |	23,095 |
| MX.02	| 16 |
| YB05 |	12 |
| MX.04 |	0 |

In [ ]:
!qiime feature-table filter-samples \
  --i-table CPY16S/table-freq-cenotes.qza \
  --m-metadata-file CPY16S/metadata.tsv \
  --o-filtered-table CPY16S/sites_filtered/table-freq-cpy16s.qza

/bin/bash: qiime: command not found


Now, the sites with few sequences (MX.02, YB05 and MX.04) were removed. Let's create a visualization artifact to confirm that

In [ ]:
!qiime feature-table summarize \
  --i-table CPY16S/sites_filtered/table-freq-cpy16s.qza\
  --o-visualization CPY16S/sites_filtered//table-freq-tes-cpy16s.qzv \
  --m-sample-metadata-file CPY16S/metadata.tsv

/bin/bash: qiime: command not found


Confirmed the sites removed, the sequences will remove too

In [ ]:
!qiime feature-table filter-seqs \
  --i-data CPY16S/rep-seqs-cenotes.qza \
  --i-table CPY16S/sites_filtered/table-freq-cpy16s.qza \
  --o-filtered-data CPY16S/sites_filtered/rep-seqs-cpy16s.qza

/bin/bash: qiime: command not found


and tabulate the sequences

In [ ]:
!qiime feature-table tabulate-seqs \
  --i-data CPY16S/sites_filtered/rep-seqs-cpy16s.qza \
  --o-visualization CPY16S/sites_filtered/rep-seqs-cpy16s.qzv

/bin/bash: qiime: command not found


__Phylogenetic diversity analyses__

building a phylogenetic tree

In [ ]:
!qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences CPY16S/sites_filtered/rep-seqs-cpy16s.qza \
    --output-dir CPY16S/tree

/bin/bash: qiime: command not found


Alpha and Beta Diversity

In [ ]:
!qiime diversity core-metrics-phylogenetic \
    --i-table CPY16S/sites_filtered/table-freq-cpy16s.qza \
    --i-phylogeny CPY16S/tree/rooted_tree.qza \
    --p-sampling-depth 23094 \
    --m-metadata-file CPY16S/metadata.tsv \
    --output-dir CPY16S/diversity

/bin/bash: qiime: command not found


**Taxonomy Analisys**

We will use a Bayes classifier trained on the GreenGenes database which can be downloaded from:
https://data.qiime2.org/2021.11/common/silva-138-99-515-806-nb-classifier.qza

In [ ]:
!wget https://data.qiime2.org/2021.11/common/silva-138-99-515-806-nb-classifier.qza

--2022-04-27 17:46:30--  https://data.qiime2.org/2021.11/common/silva-138-99-515-806-nb-classifier.qza
Resolving data.qiime2.org (data.qiime2.org)... 54.200.1.12
Connecting to data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2021.11/common/silva-138-99-515-806-nb-classifier.qza [following]
--2022-04-27 17:46:31--  https://s3-us-west-2.amazonaws.com/qiime2-data/2021.11/common/silva-138-99-515-806-nb-classifier.qza
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.233.96
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.233.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 148294965 (141M) [binary/octet-stream]
Saving to: ‘silva-138-99-515-806-nb-classifier.qza’

silva-138-99-515-80 100%[===================>] 141.42M  41.8MB/s    in 3.4s    

2022-04-27 17:46:34 (41.8 MB/s) - ‘silva-138-9

In [ ]:
!qiime feature-classifier classify-sklearn \
    --i-reads CPY16S/sites_filtered/rep-seqs-cpy16s.qza \
    --i-classifier silva-138-99-515-806-nb-classifier.qza \
    --o-classification CPY16S/taxonomy/taxa-cpy16s.qza

/bin/bash: qiime: command not found


Now let's have a look at the relative abundances of the different bacterial taxa we have in each sample:

In [ ]:
!qiime taxa barplot \
    --i-table dada2/table.qza \
    --i-taxonomy taxa.qza \
    --m-metadata-file metadata.tsv \
    --o-visualization taxa_barplot.qzv

/bin/bash: qiime: command not found


In [ ]:
import sklearn
sklearn.__version__
sklearn.show_versions()


System:
    python: 3.7.13 (default, Apr 24 2022, 01:04:09)  [GCC 7.5.0]
executable: /usr/bin/python3
   machine: Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic

Python dependencies:
          pip: 21.1.3
   setuptools: 57.4.0
      sklearn: 1.0.2
        numpy: 1.21.6
        scipy: 1.4.1
       Cython: 0.29.28
       pandas: 1.3.5
   matplotlib: 3.2.2
       joblib: 1.1.0
threadpoolctl: 3.1.0

Built with OpenMP: True


The scikit-learn version (0.24.1) used to generate this artifact does not match the current version of scikit-learn installed (0.22.1).